In [11]:
# Import libraries
import pandas as pd
import sys, os
import matplotlib.pyplot as plt

import folium
from folium import plugins

from wordcloud import WordCloud, STOPWORDS

from transformers import pipeline

In [ ]:
# Function for output csv file
pd.options.mode.copy_on_write = True


def data_exploration(df_combined, months_folders, output_csv_path, columns_to_select):

    # Number of NaN values replaced
    nan_replaced_count = 0

    # For each month
    for month_folder in months_folders:
        # Start by importing "listings.csv"
        input_csv_path = os.path.join(month_folder, 'listings.csv')
        
        # The dataframe
        df = pd.read_csv(input_csv_path, low_memory=False)
        
        # Include only the columns that we want and are in the .csv file
        columns_to_keep = [col for col in columns_to_select if col in df.columns]
        df_selected = df[columns_to_keep]
        
        # Columns from other .csv files
        additional_columns = set()
        
        # Check all other .csv files in the folder
        for filename in os.listdir(month_folder):
            if filename.endswith('.csv') and filename != 'listings.csv':  # We want .csv files but no the starting one, aka "listings.csv"
                file_path = os.path.join(month_folder, filename)
                other_df = pd.read_csv(file_path)

                # Iterate over each column
                for col in other_df.columns:
                    # If we find a column that we want but is not already in our dataframe, add it from the other .csv file
                    if col not in df_selected.columns and col in columns_to_select:
                        additional_columns.add(col)
                        # Add the column and its data to our dataframe
                        df_selected[col] = other_df[col]

                    elif  col in df_selected.columns and col in columns_to_select:
                        # Check if any value of the column in our dataframe is NaN
                        nan_mask = df_selected[col].isna()
                        
                        # Check if the corresponding value in the other dataframes is not NaN
                        non_nan_mask = ~other_df[col].isna()
                        
                        # Replace NaN values with not NaN ones
                        df_selected.loc[nan_mask & non_nan_mask, col] = other_df.loc[nan_mask & non_nan_mask, col]

                        # Number of NaN values replaced
                        nan_replaced_count += sum(nan_mask & non_nan_mask)

        # Concatenate every months dataframes
        df_combined = pd.concat([df_combined, df_selected], ignore_index=True)

    # Write the final dataframe to the output .csv file
    df_combined.to_csv(output_csv_path, index=False)

    print(f"Data from all months has been written to {output_csv_path}")
    print(f"The output CSV file has {df_selected.shape[1]} columns.")
    print(f"Number of NaN values replaced: {nan_replaced_count}")

    return df_combined

# Initialize a DataFrame to store data for 2019
df_combined_2019 = pd.DataFrame()

# Initialize a DataFrame to store data for 2023
df_combined_2023 = pd.DataFrame()

# Columns we want to select
columns_to_select_2019 = ['id', 'zipcode', 'transit', 'bedrooms', 'beds', 'review_scores_rating', 'number_of_reviews',
                        'neighbourhood', 'name', 'latitude', 'longitude', 'last_review', 'instant_bookable', 'host_since',
                        'host_response_rate','host_identity_verified', 'host_has_profile_pic', 'first_review', 'description','city',
                        'cancellation_policy', 'bed_type', 'bathrooms', 'accommodates', 'amenities', 'room_type',
                        'property_type', 'price', 'availability_365', 'minimum_nights', 'last_scraped', 'comments', 'host_id', 'host_name']

columns_to_select_2023 = ['id', 'zipcode', 'transit', 'bedrooms', 'beds', 'review_scores_rating', 'number_of_reviews',
                        'neighbourhood_cleansed', 'name', 'latitude', 'longitude', 'last_review', 'instant_bookable', 'host_since',
                        'host_response_rate','host_identity_verified', 'host_has_profile_pic', 'first_review', 'description','city',
                        'cancellation_policy', 'bed_type', 'bathrooms', 'accommodates', 'amenities', 'room_type',
                        'property_type', 'price', 'availability_365', 'minimum_nights', 'last_scraped', 'comments', 'host_id', 'host_name']

### 2019 ###

# Each month's folder
months_folders = ['data/2019/april', 'data/2019/febrouary', 'data/2019/march']

# Path to the output CSV file
output_csv_path_2019 = 'data_train/train_2019.csv'

print("For 2019:")
df_combined_2019 = data_exploration(df_combined_2019, months_folders, output_csv_path_2019, columns_to_select_2019)

### 2023 ###

# Each month's folder
months_folders = ['data/2023/june', 'data/2023/march', 'data/2023/september']

# Path to the output CSV file
output_csv_path_2023 = 'data_train/train_2023.csv'

print("\nFor 2023:")
df_combined_2023 = data_exploration(df_combined_2023, months_folders, output_csv_path_2023, columns_to_select_2023)
df_combined_2023.rename(columns={'neighbourhood_cleansed': 'neighbourhood'}, inplace=True)

In [ ]:
# data cleaning 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def cleaning_data(df):
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ', 'Plaka')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ', 'Attiki')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ', 'Exarcheia')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ','Agios Konstantinos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΥΨΕΛΗ', 'Kypseli')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΑΓΚΡΑΤΙ', 'Pangrati')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΝΕΟΣ ΚΟΣΜΟΣ', 'Neos Kosmos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΟΛΩΝΑΚΙ', 'Kolonaki')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΜΠΕΛΟΚΗΠΟΙ', 'Ambelokipi')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ', 'Koukaki')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΕΡΑΜΕΙΚΟΣ', 'Kerameikos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΣΤΑΔΙΟ', 'Stadio')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΚΡΟΠΟΛΗ', 'Akropoli')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΛΥΚΑΒΗΤΤΟΣ', 'Lykavittos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΕΝΤΑΓΩΝΟ', 'Pedagono')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ', 'Akadimia Platonos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΘΗΣΕΙΟ', 'Thiseio')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΓΟΥΒΑ', 'Gouva')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΓΟΥΔΙ', 'Goudi')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΕΔΙΟ ΑΡΕΩΣ', 'Pedion Areos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΝΩ ΠΑΤΗΣΙΑ', 'Patisia')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΟΛΩΝΟΣ', 'Kolonos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΝΙΡΒΑΝΑ', 'Nirvana')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΕΛΛΗΝΟΡΩΣΩΝ', 'Ellinoroson')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΙΛΙΣΙΑ', 'Ilisia')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΖΑΠΠΕΙΟ', 'Zappeio')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΓΚΥΖΗ', 'Gizi')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΒΟΤΑΝΙΚΟΣ', 'Votanikos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ', 'Agios Nikolaos')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΕΤΡΑΛΩΝΑ', 'Petralona')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΓΚΑΖΙ', 'Gazi')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('1Ο ΝΕΚΡΟΤΑΦΕΙΟ', '1o Nekrotafio')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΑΤΗΣΙΑ', 'Patisia')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΡΟΜΠΟΝΑ', 'Probona')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΟΛΥΓΩΝΟ', 'Poligono')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ','Agios Eleftherios')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ', 'Plateia Amerikis')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΚΟΛΟΚΥΝΘΟΥ', 'Kolokinthou')
    
    # specifically for 2023
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΑΝΩ ', '')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ', 'Stathmos Larisis')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΝΕΑ ', '')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΣΕΠΟΛΙΑ', 'Sepolia')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΡΙΖΟΥΠΟΛΗ', 'Rizoupoli')
    df['neighbourhood'] = df['neighbourhood'].astype(str).str.replace('ΡΗΓΙΛΛΗΣ', 'Rigilis')


    # general cleaning
    df['last_scraped'] = pd.to_datetime(df['last_scraped'])
    # Extract month from date
    df['month'] = df['last_scraped'].dt.month.astype('category')
    # Clean the 'price' column by removing commas and dollar signs
    df['price'] = df['price'].astype(str).str.replace(',', '').str.replace('$', '')
    # Convert the 'price' column to float
    df['price'] = df['price'].astype(float)

    # NaN cleaning
    if 'zipcode' in df.columns:
        df['zipcode'] = df['zipcode'].fillna('-')
    if 'transit' in df.columns:
        df['transit'] = df['transit'].fillna('-')
    if 'bedrooms' in df.columns:
        df['bedrooms'] = df['bedrooms'].fillna(0)
    if 'beds' in df.columns:
        df['beds'] = df['beds'].fillna(0)
    if 'name' in df.columns:
        df['name'] = df['name'].fillna('-')
    if 'last_review' in df.columns:
        df['last_review'] = df['last_review'].fillna(0)
    if 'host_since' in df.columns:
        df['host_since'] = df['host_since'].fillna(0)
    if 'host_response_rate' in df.columns:
        df['host_response_rate'] = df['host_response_rate'].fillna('0%')
    if 'host_identity_verified' in df.columns:
        df['host_identity_verified'] = df['host_identity_verified'].fillna('f')
    if 'host_has_profile_pic' in df.columns:
        df['host_has_profile_pic'] = df['host_has_profile_pic'].fillna('f')
    if 'first_review' in df.columns:
        df['first_review'] = df['first_review'].fillna(0)
    if 'city' in df.columns:
        df['city'] = df['city'].fillna('Athens')
    if 'bathrooms' in df.columns:
        df['bathrooms'] = df['bathrooms'].fillna(0)
    if 'host_name' in df.columns:
        df['host_name'] = df['host_name'].fillna('NO NAME')

    return df


# Clean 2019
dataframe_2019 = cleaning_data(df_combined_2019)
# Write output csv
dataframe_2019.to_csv(output_csv_path_2019, index=False)

# Clean 2023
dataframe_2023 = cleaning_data(df_combined_2023)
indexx = dataframe_2023[(dataframe_2023['month'] == 4)].index   # Drop April from 2023 data because of missinformation
dataframe_2023.drop(indexx, inplace=True)
# Write output csv
dataframe_2023.to_csv(output_csv_path_2023, index=False)
